# Google Natural Questions

## Introduction

Le but de ce challenge est de créer un modèle d'apprentissage statistique qui trouve des réponses posées à un corps de texte. 

## Présentation des données

Dans cette partie nous présentons les données fournies. Pour voir une version tabulée des données, vous pouvez consulter ce [lien](https://ai.google.com/research/NaturalQuestions/databrowser).

Nous utiliserons les données simplifiées par Google:

- **simplified-nq-train.jsonl**: 17.45 Go
- **simplified-nq-test.jsonl**: 18.8 Mo

La base d'apprentissage étant assez volumineuse, nous utiliserons le Google Cloud Platform.

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

local = "local[*]"
appName = "QA"
configLocale = SparkConf().setAppName(appName).setMaster(local).\
set("spark.executor.memory", "6G").\
set("spark.driver.memory", "6G").\
set("spark.sql.catalogImplementation", "in-memory")

spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
spark

Afin d'inspecter le format du fichier `json`nous avons créé une version courte, que nous chargeons ici.

In [7]:
for_content = spark.read.json("/Volumes/750GB-HDD/root/Question-Answering/pyData/tensorflow2-question-answering/simplified-nq-train-for-content.jsonl")

for_content.printSchema()

root
 |-- annotations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotation_id: long (nullable = true)
 |    |    |-- long_answer: struct (nullable = true)
 |    |    |    |-- candidate_index: long (nullable = true)
 |    |    |    |-- end_token: long (nullable = true)
 |    |    |    |-- start_token: long (nullable = true)
 |    |    |-- short_answers: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end_token: long (nullable = true)
 |    |    |    |    |-- start_token: long (nullable = true)
 |    |    |-- yes_no_answer: string (nullable = true)
 |-- document_text: string (nullable = true)
 |-- document_url: string (nullable = true)
 |-- example_id: long (nullable = true)
 |-- long_answer_candidates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- end_token: long (nullable = true)
 |    |    |-- start_token: long (nullable = true)
 |    |    |

In [9]:
for_content.show()

+--------------------+--------------------+--------------------+-------------------+----------------------+--------------------+
|         annotations|       document_text|        document_url|         example_id|long_answer_candidates|       question_text|
+--------------------+--------------------+--------------------+-------------------+----------------------+--------------------+
|[[593165450220027...|Email marketing -...|https://en.wikipe...|5655493461695504401|  [[170, 14, true],...|which is the most...|
+--------------------+--------------------+--------------------+-------------------+----------------------+--------------------+

